In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
import pack_ga
from IPython.display import HTML, display, clear_output
kgs.set_float32(True)


vast
vast
stop final relax at some point


In [2]:
%load_ext pyinstrument

In [4]:
importlib.reload(pack_ga)
fastMode = False

# fast for now
#runner.population_size = 1
#runner.selection_size = [0]
#runner.n_generations = 500

# structural
ratios = np.arange(0.4,0.6,0.01)
res = []
for r in ratios:
    print(r)
    runner = pack_ga.GA()
    runner.N_trees_to_do = np.array([2])
    runner.population_size = 10000
    runner.plot_fitness_predictors = False
    runner.plot_diversity_matrix = False
    runner.plot_champions = False
    runner.initializer.base_solution = kgs.SolutionCollectionLatticeFixed()
    runner.initializer.base_solution.aspect_ratios = cp.array([1/r], dtype=kgs.dtype_cp)
    runner.initializer.jiggler.duration_init = 0.
    runner.initializer.jiggler.duration_compact /= 100000
    runner.initializer.jiggler.duration_final = 0.
    runner.initializer.jiggler.plot_interval = None
    runner.initializer.jiggler.n_rounds = 1
    runner.rough_relaxers[0].plot_interval =None
    runner.rough_relaxers[0].n_iterations//=5
    runner.fine_relaxers[0].plot_interval=None
    runner.rough_relaxers[0].cost.costs.pop(1)
    runner.fine_relaxers[0].cost.costs.pop(1)
    runner.fine_relaxers[0].n_iterations//=5
    runner.fine_relaxers.append(copy.deepcopy(runner.fine_relaxers[0]))
    runner.fine_relaxers[1].cost.costs[0].scaling/=2
    runner.fine_relaxers.append(copy.deepcopy(runner.fine_relaxers[0]))
    runner.fine_relaxers[1].cost.costs[0].scaling/=4
    runner.n_generations = 5
    runner.fitness_cost.costs.pop(1)
    kgs.debugging_mode = 1
    kgs.profiling=False
    new_moves = []
    for i in [1,2]:
        new_moves.append(runner.move.moves[i])
    runner.move.moves = new_moves
    #print(runner.move.moves)
    #runner.rough_relaxers = []
    #runner.fine_relaxers = []
    runner.run()
    res.append(runner)

stop final relax at some point
0.4
Generation 0, Trees 2, Best cost: 0.00888088, Est: 0.44404402, h: 0.590704
[[0.5907043]]
Generation 1, Trees 2, Best cost: 0.00716050, Est: 0.35802513, h: 0.531938
[[0.5319382]]
Generation 2, Trees 2, Best cost: 0.00716050, Est: 0.35802481, h: 0.531914
[[0.53191364]]
Generation 3, Trees 2, Best cost: 0.00716049, Est: 0.35802463, h: 0.531893
[[0.5318931]]
Generation 4, Trees 2, Best cost: 0.00716049, Est: 0.35802463, h: 0.531893
[[0.5318931]]
0.41000000000000003
Generation 0, Trees 2, Best cost: 0.00871738, Est: 0.43586880, h: 0.593236
[[0.5932358]]
Generation 1, Trees 2, Best cost: 0.00698795, Est: 0.34939772, h: 0.532060
[[0.5320601]]
Generation 2, Trees 2, Best cost: 0.00698795, Est: 0.34939760, h: 0.532052
[[0.53205204]]
Generation 3, Trees 2, Best cost: 0.00698795, Est: 0.34939754, h: 0.532040
[[0.5320397]]
Generation 4, Trees 2, Best cost: 0.00698795, Est: 0.34939754, h: 0.532029
[[0.5320292]]
0.42000000000000004
Generation 0, Trees 2, Best cost:

In [5]:
res_pop = []
for r in res:
    pop = copy.deepcopy(r.populations[0])
    pop.select_ids([np.argmin(pop.fitness)])
    res_pop.append(pop.configuration)
pop = res_pop[0]
for r in res_pop[1:]:
    pop.merge(r)
kgs.dill_save(kgs.code_dir + '../res/packings.pickle', pop)